In [1]:
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree

import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.patches import Patch
import os
from shutil import copy
import seaborn as sns

import datetime
from dateutil.relativedelta import relativedelta
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import imageio
from itertools import compress


pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams.update({'figure.max_open_warning': 0})

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'xx-large',
          'legend.title_fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
#‘xx-small’, ‘x-small’, ‘small’, ‘medium’, ‘large’, ‘x-large’, ‘xx-large’
pylab.rcParams.update(params)
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
# from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
from statsmodels.tsa.api import VAR

from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn import tree

import re

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.optimize import curve_fit
from tslearn.clustering import KShape, silhouette_score

import gc
import matplotlib.dates as mdates
from sklearn.ensemble import IsolationForest
import math
import warnings
from scipy import stats
from tqdm import tqdm


def weighted_avg(x, weight, factor):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        tmp = x[[weight, factor]].dropna()
        weighted_sum = (tmp[weight] * tmp[factor]).sum()
        count_sum = tmp[weight].sum()
        return weighted_sum / count_sum

#read data
locus_weights=pd.read_csv('..\\data\\evt_movement_ratio_with_dates.csv')
locus_weights.starttime = pd.to_datetime(locus_weights.starttime,format='%Y-%m-%d')
locus_weights.endtime = pd.to_datetime(locus_weights.endtime,format='%Y-%m-%d')

temperature = pd.read_csv('..\\data\\temperature_for_CAM.csv')
temperature.event_date = pd.to_datetime(temperature.event_date,format='%Y-%m-%d')
temperature.locus_group_id=temperature.locus_group_id.astype('int16')
#not sure in row below as it 'converts' 12.3 -> 12.296875
temperature.value=temperature.value.astype('float16')
temperature['event_year']=temperature['event_date'].dt.year

llg_match= pd.read_csv('..\\data\\locus_locus_group_matching.csv')
llg_match.locus_id=llg_match.locus_id.astype('int32')
llg_match.locus_group_id=llg_match.locus_group_id.astype('int16')

df_dates=pd.read_csv('..\\data\\FW_cycle_dates.csv')
for d in ['first_movement_date', 'first_feeding_date', 'shipout_date']:
    df_dates[d] = pd.to_datetime(df_dates[d],format='%Y-%m-%d')
    
sfm = pd.read_csv('..\\data\\seawater_freshwater_matching.csv')
sfm_ = sfm[sfm.origin_site_type=='Freshwater'][['target_seawater_locus_id','transport_date','ponding_date','pretransfer_fw_locus_population_id','fish_count_shipped_out','avg_weight_g_stocked']]
sfm_.pretransfer_fw_locus_population_id=sfm_.pretransfer_fw_locus_population_id.astype('int64')


AttributeError: 'DataFrame' object has no attribute 'locus_group_id'

In [ ]:
lw_dates=locus_weights.groupby('final_locus_population_id').agg({'starttime':'min','endtime':'max'})
lw_dates.starttime = pd.to_datetime(lw_dates.starttime,format='%Y-%m-%d')
lw_dates.endtime = pd.to_datetime(lw_dates.endtime,format='%Y-%m-%d')
#to be checked
lw_dates['FW_cycle_length'] = (lw_dates.endtime - lw_dates.starttime).dt.days+1
lw_dates['starttime_year']=lw_dates['starttime'].dt.year
#we limit FW cycles to those started in 2017 because there are issues with temperature readings for 2015-2016
lw_dates_2017=lw_dates[lw_dates.starttime_year>=2017]

N = len(locus_weights['final_locus_population_id'].unique())
rnd_idxs = np.random.randint(0, N, 50)
mask = locus_weights['final_locus_population_id'].unique()[rnd_idxs]
mask = locus_weights['final_locus_population_id'].isin(mask)

#time consuming
lw_alldates_list = []
for ind, row in tqdm(locus_weights[mask].iterrows()):
    for d in pd.date_range(row.starttime,row.endtime-datetime.timedelta(days=1)):
        lw_alldates_list.append([row.final_locus_population_id,d,row.historic_locus_id,row.count_ratio])
lw_alldates = pd.DataFrame(lw_alldates_list, columns = ['final_locus_population_id','event_date','historic_locus_id','weight0'])
lw_alldates_weights_grouped=lw_alldates.groupby(['final_locus_population_id','event_date'])[['weight0']].sum().reset_index()
lw_alldates_weights_grouped_merged=lw_alldates.merge(lw_alldates_weights_grouped, on=['final_locus_population_id','event_date'], how='left')
lw_alldates_weights_grouped_merged['weight']=lw_alldates_weights_grouped_merged['weight0_x']/lw_alldates_weights_grouped_merged['weight0_y']
lw_alldates_final=lw_alldates_weights_grouped_merged[['final_locus_population_id', 'event_date', 'historic_locus_id','weight']].sort_values(by=['final_locus_population_id','event_date','historic_locus_id'])
lw_alldates_final.historic_locus_id=lw_alldates_final.historic_locus_id.astype('int32')

def expand_dates_vectorized(df):
    df = df.loc[df.index.repeat((df.endtime - df.starttime).dt.days)]
    df['event_date'] = df.groupby(level=0)['starttime'].transform(lambda x: x + pd.to_timedelta(np.arange(len(x)), 'D'))
    df = df.drop(columns=['starttime', 'endtime'])
    df = df.rename({'count_ratio': 'weight0'}, axis=1)
    return df

lw_alldates_final = []
for lp_id, lp_df in tqdm(locus_weights.groupby('final_locus_population_id'), 'Assigning LP weights'):
    e_df = expand_dates_vectorized(lp_df)
    # e_df = pd.concat([expand_dates(row) for _, row in lp_df.iterrows()], ignore_index=True)
    agg_df = e_df.groupby(['event_date', 'final_locus_population_id']).sum().reset_index()
    agg_df_merged=e_df.merge(agg_df, on=['final_locus_population_id','event_date', 'historic_locus_id'], how='left')
    agg_df_merged['weight']=agg_df_merged['weight0_x']/agg_df_merged['weight0_y']
    res_df = agg_df_merged[['final_locus_population_id', 'event_date', 'historic_locus_id','weight']].sort_values(by=['final_locus_population_id','event_date','historic_locus_id'])
    res_df.historic_locus_id=res_df.historic_locus_id.astype('int32')

    lw_alldates_final.append(res_df)
lw_alldates_final = pd.concat(lw_alldates_final, axis=0)
lw_alldates_final.to_csv('../data/lw_alldates_final.csv', index=False)


In [10]:
lw_alldates_final = pd.read_csv('../data/lw_alldates_final.csv')
lw_alldates_final['event_date'] = pd.to_datetime(lw_alldates_final['event_date'])

In [11]:
lw_alldates_final_=lw_alldates_final.merge(llg_match,left_on='historic_locus_id', right_on='locus_id', how='left')
#alternatively rename column before merging. Thus not having to drop column thereafter
lw_alldates_final_.drop(columns='locus_id',inplace=True)
lw_alldates_final=[]

In [12]:
# gc.collect()
lw_alldates_final__=lw_alldates_final_.merge(temperature[['locus_group_id', 'event_date', 'value']],how='left')
lw_alldates_final__['event_year']=lw_alldates_final__['event_date'].dt.year
# lw_alldates_final__.groupby('event_year').value.count()
#30% of dates were not matched
#temperature readings for 2015 and 2016 are absent in evt_logbook
#3% of cages were not matched

#removed absent records for temperature and values above 20 degrees - this filter also removes nan
#add coverage here? to understand how much of temperature data is missed
#alternatively can replace NaN with some average value
lw_alldates_final__=lw_alldates_final__[(lw_alldates_final__.value<20) | (lw_alldates_final__.value.isna())]
lw_alldates_final__.rename(columns={'value':'temperature'},inplace=True)
lw_alldates_final__.temperature=lw_alldates_final__.temperature.astype('float32').round(1)
lw_alldates_final__['weight_temperature']=lw_alldates_final__['weight']*lw_alldates_final__['temperature']

In [13]:
# lw_alldates_final_grouped=lw_alldates_final__.groupby(['final_locus_population_id','event_date'])[['weight_temperature']].sum().reset_index()
lw_alldates_final_grouped=lw_alldates_final__.groupby(['final_locus_population_id','event_date'])[['weight_temperature']].agg(lambda x: x.sum(skipna=False)).reset_index()

#len(lw_alldates_final_grouped.final_locus_population_id.unique())/len(locus_weights.final_locus_population_id.unique())
#after temperature processing 6% of final_locus_population_id have been lost
lw_alldates_final_grouped.rename(columns={'weight_temperature':'temperature'},inplace=True)
# lw_alldates_final_grouped['temperature']=lw_alldates_final_grouped['temperature'].round(1).astype('str')
lw_alldates_final_grouped.to_csv('..//data//lw_alldates_final_grouped.csv',index=False)

OSError: Cannot save file into a non-existent directory: 'data'